# MLP w TensorFlow

Najpierw musimy zaimportować bibliotekę tensorflow. 


In [10]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

Następnie musimy określić liczbę wejść i wyjść oraz ustawić liczbę ukrytych neuronów w każdej warstwie

In [4]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

Następnie, można użyć węzłów typu placeholder do reprezentowania danych treningowych. 

  * Kształt $X$ jest tylko częściowo zdefiniowany. 
  * Wiemy, że będzie to tensor 2D, z instancjami wzdłuż pierwszego wymiaru i cechami wzdłuż drugiego wymiaru, i wiemy, że liczba funkcji będzie wynosić $28 x 28$ (jedna cecha na piksel), ale nie wiemy jeszcze, ile wystąpi elementów. 
  * Zatem kształt $X$ to (None, n_inputs)
  * Podobnie, wiemy, że $y$ będzie tensorem 1D z jedą kolumną na instancję, ale znowu nie znamy wielkości zbioru treningowego, więc kształt jest (None).

In [5]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

# Teraz stwórzmy właściwą sieć neuronową. 

  * Placeholder $X$ będzie działał jako warstwa wejściowa; 
  * w fazie wykonania do placecholdera zastanie podana jedna część (batch) zbioru treningowego (należy pamiętać, że wszystkie elementy batcha będą przetwarzane jednocześnie przez sieć neuronową). 

Teraz musisz utworzyć dwie ukryte warstwy i warstwę wyjściową. Dwie ukryte warstwy są prawie identyczne: 

  * różnią się tylko wejściami, z którymi są połączone, 
  * oraz liczbą neuronów, które zawierają. 

Warstwa wyjściowa jest również bardzo podobna, ale wykorzystuje funkcję aktywacji softmax zamiast funkcji aktywacji ReLU.

Stwórzmy więc funkcję  **neuron_layer()**, której użyjemy do stworzenia jednej warstwy. 
Będziemy potrzebować parametrów do określenia 
  * wejść, 
  * liczby neuronów, 
  * funkcji aktywacji 
  * nazwy warstwy:

In [8]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

# Przejdźmy przez ten kod linia po linii:

  * Najpierw tworzymy zakres nazw używając nazwy warstwy: będzie on zawierał wszystkie węzły dla tej warstwy neuronu. 
  Jest to opcjonalne, ale wykres będzie wyglądał znacznie ładniej w TensorBoard.
  * Następnie odczytujemy liczbę wejść (wymiar danych) poprzez sprawdzenie kształtu macierzy wejściowej (zerowy wymiar dotyczy instancji - ilościc danych).
  * Następne trzy linie tworzą zmienną $W$, która będzie zawierała macierz wag. Będzie to tensor 2D zawierający wszystkie wagi połączeń między każdym wejściem a każdym neuronem - stąd jego kształt będzie (n_inputs, n_neurons). Zostanie zainicjalizowany losowo, przy użyciu rokładu **truncated normal**
  * truncated normal to rozkład normalny z odchyleniem standardowym $2/\sqrt{n_{inputs}}$. Wykorzystanie tego specyficznego odchylenia standardowego pomaga algorytmowi zbiegać znacznie szybciej. Ważne jest, aby inicjalizować losowe wagi dla wszystkich ukrytych warstw tak aby uniknąć symetrii, której algorytm Gradient Descent nie byłby w stanie zlikwidować.
  * Następny wiersz tworzy zmienną dla bias, zainicjowaną na 0 (w tym przypadku nie ma problemu z symetrią).
  * Następnie tworzymy podgraf aby obliczyć 
  $$
  z = X · W + b.
  $$
  * Na koniec, jeśli parametr aktywacji jest ustawiony na "relu", kod zwraca $relu(z)$ lub w przeciwnym razie zwraca tylko $z$.

# Tworzenia głębokiej sieci neuronowej

Okej, więc teraz mamy funkcję tworzącą warstwy neuronów. Wykorzystajmy go do stworzenia głębokiej sieci neuronowej! 

 * Pierwsza ukryta warstwa bierze $X$ jako swoje wejście.
 * Drugi pobiera dane wyjściowe pierwszej ukrytej warstwy jako dane wejściowe. 
 * Na koniec, warstwa wyjściowa pobiera dane wyjściowe drugiej ukrytej warstwy jako dane wejściowe.

In [11]:
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_outputs, "outputs")

  * Zauważ, że ponownie użyliśmy zakresu nazw. 
  * Zauważ również, że **logits** są danymi wyjściowymi sieci neuronowej przed przejściem przez funkcję aktywacji softmax: 
    * ze względów optymalizacyjnych zajmiemy się później obliczaniem softmax.

# Funkcja kosztu

  * Teraz, gdy mamy gotowy model sieci neuronowej musimy zdefiniować funkcję kosztu, której użyjemy do jej uczenia. 
  * Podobnie jak w przypadku Regresji Softmax, użyjemy entropii krzyżowej. 
  * Entropia krzyżowa kara modele, który daje niskie prawdopodobieństwo dla klasy docelowej. 
  * TensorFlow udostępnia kilka funkcji do obliczania entropii krzyżowej. 
  * Użyjemy **sparse_softmax_cross_entropy_with_logits()**: 
    * oblicza entropię krzyżową na podstawie "logits" (tj. Wyjścia sieci przed przejściem przez funkcję aktywacji softmax) i oczekuje etykiet w postaci liczb całkowitych od 0 do liczby klas minus 1 (w naszym przypadku od 0 do 9). 
    * Zwróci ona tensor 1D zawierający entropię krzyżową dla każdej instancji. 
    
    
  * Możemy następnie użyć funkcji TensorFlow **redu_mean()**, aby obliczyć średnią wartość entropii krzyżowej we wszystkich instancjach.

In [12]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss") 

# Optymalizacja
Mamy już model sieci neuronowej oraz funkcję kosztu. 
Teraz musimy zdefiniować GradientDescentOptimizer, który znajdzie parametry modelu które minimalizują funkcję kosztu.

In [13]:
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)  

# Sposób oceny modelu
  * Ostatnim ważnym krokiem jest określenie sposobu oceny modelu. 
  * Po prostu użyjemy **accuracy** jako naszej miary wydajności.
    * Po pierwsze dla każdej instancji określ, czy prognoza sieci neuronowej jest prawidłowa, sprawdzając, czy najwyższy logit odpowiada klasie docelowej.
    * Do tego możesz użyć funkcji **in_top_k()**. Zwraca ona tensor 1D wartości boolowskich, więc musimy zrzutować te wartości logiczne na wartości zmiennoprzecinkowe, a następnie obliczyć średnią. To da nam ogólną dokładność sieci.

In [14]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))   

# Inicjalizacja
Musimy utworzyć węzeł, aby zainicjować wszystkie zmienne:

In [15]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# Uczenie

## Najpierw załadujmy MNIST. 

Moglibyśmy używać Scikit-Learn do tego, tak jak to robiliśmy w poprzednich rozdziałach, ale TensorFlow oferuje własną funkcję, który pobiera dane, skaluje je (między 0 a 1), miesza je i zapewnia prostą funkcję ładowania jednego mini batcha. 
Więc użyjmy tego:

In [16]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


Teraz definiujemy liczbę epok, które chcemy uruchomić, a także rozmiar batch-a:

In [27]:
n_epochs = 100
batch_size = 64

A teraz możemy wytrenować model:

In [29]:
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(mnist.train.num_examples // batch_size):
            print("\r{}%".format(100 * batch_index //  (mnist.train.num_examples // batch_size) ), end="")
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(" ", epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)    
        
    save_path = saver.save(sess, "./MNIST_MLP/my_model_final.ckpt")   

99%  0 Train accuracy: 0.9375 Test accuracy: 0.9065
99%  1 Train accuracy: 0.96875 Test accuracy: 0.9266
99%  2 Train accuracy: 0.9375 Test accuracy: 0.9358
99%  3 Train accuracy: 0.953125 Test accuracy: 0.9435
99%  4 Train accuracy: 0.953125 Test accuracy: 0.9458
99%  5 Train accuracy: 0.890625 Test accuracy: 0.952
99%  6 Train accuracy: 0.921875 Test accuracy: 0.9534
99%  7 Train accuracy: 0.9375 Test accuracy: 0.9551
99%  8 Train accuracy: 0.96875 Test accuracy: 0.9575
99%  9 Train accuracy: 0.9375 Test accuracy: 0.957
99%  10 Train accuracy: 0.984375 Test accuracy: 0.961
99%  11 Train accuracy: 0.984375 Test accuracy: 0.9629
99%  12 Train accuracy: 0.921875 Test accuracy: 0.9637
99%  13 Train accuracy: 0.984375 Test accuracy: 0.9648
99%  14 Train accuracy: 0.96875 Test accuracy: 0.965
99%  15 Train accuracy: 0.96875 Test accuracy: 0.967
99%  16 Train accuracy: 1.0 Test accuracy: 0.9677
99%  17 Train accuracy: 0.953125 Test accuracy: 0.9679
99%  18 Train accuracy: 0.96875 Test accur

  * Ten kod otwiera sesję TensorFlow i uruchamia węzeł inicjujący, który inicjuje wszystkie zmienne. 
  * Następnie uruchamia główną pętlę treningową: w każdej epoce kod iteruje przez kilka batch-y odpowiadających rozmiarowi zestawu treningowego. 
  * Każda batch jest pobierana za pomocą metody **next_batch()**, a następnie kod po prostu uruchamia operację uczenia.   *   * 
  * Następnie, na końcu każdej epoki, kod ocenia model na ostatnim batch-u i na pełnym zestawie treningowym.

# Po nauczeniu sieci neuronowej można jej użyć do przewidywania. 

In [40]:
with tf.Session() as sess:
    saver.restore(sess, "./MNIST_MLP/my_model_final.ckpt") 
    X_new_scaled, y_true = mnist.train.next_batch(1) # some new images (scaled from 0 to 1)
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)
print("True label ", y_true)
print("Predicted label ", y_pred)

INFO:tensorflow:Restoring parameters from ./MNIST_MLP/my_model_final.ckpt
True label  [6]
Predicted label  [6]


# Uwaga! 

Istnieje również inna funkcja o nazwie **softmax_cross_entropy_with_logits()**, która pobiera etykiety w postaci **one-hot vectors ** (zamiast liczby od 0 do liczby klas minus 1).